In [ ]:
pip install datasets wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.

In [ ]:
pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-yvrhfdzl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-yvrhfdzl
  Resolved https://github.com/huggingface/accelerate to commit e6699e6aba9a0900dc2dc42f5832837af4a1be9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=225536 sha256=13ee3d703db8111461d343370cf25a0cf06b2638c75c4973831c7ff8f471e742
  Stored in directory: /tmp/pip-ephem-wheel-cache-pm8crvqb/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from tqdm import tqdm

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Read the CSV dataframe
df = pd.read_csv("Data/corrected_data.csv")



df = df.sample(frac=1, random_state=42)

# df = df.sample(frac=0.1)
# Split the dataframe into train, val, and test splits
train_df = df.sample(frac=0.8)
val_df = df.drop(train_df.index)
test_df = val_df.sample(frac=0.5)

# Create the hugging face dataset
dataset = DatasetDict()
dataset["train"] = Dataset.from_pandas(train_df)
dataset["val"] = Dataset.from_pandas(val_df)
dataset["test"] = Dataset.from_pandas(test_df)

In [ ]:
model_name = "Bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
def preprocess(examples):
    max_length = 128
    return tokenizer(
        examples["Content"], truncation=True, padding="max_length", max_length=max_length
    )

In [ ]:
dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/30428 [00:00<?, ? examples/s]

Map:   0%|          | 0/7607 [00:00<?, ? examples/s]

Map:   0%|          | 0/3804 [00:00<?, ? examples/s]

In [ ]:
import wandb
import numpy as np
import torch

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Content', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30428
    })
    val: Dataset({
        features: ['Content', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7607
    })
    test: Dataset({
        features: ['Content', 'labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3804
    })
})


In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at Bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def objective(model, weight_decay, num_train_epochs, train_batch, eval_batch, optimizer, lr_scheduler):

    device = torch.device('cuda')
    model.to(device)
    training_args = TrainingArguments(
        output_dir="mBert-test-exp1/",
        weight_decay=weight_decay,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=train_batch,
        per_device_eval_batch_size=eval_batch,
        gradient_accumulation_steps= 4,
#         disable_tqdm=True,
        report_to = 'wandb'
    )
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["val"],
        optimizers = (optimizer,lr_scheduler),
    )

    wandb.init(project="Unmasking-Hate", name="mbert-tel-run-custom-1")

    trainer.train()

    # Evaluate the model after each epoch.
    progress_bar = tqdm(range(training_args.num_train_epochs), desc="Epochs", position=0)
    for epoch in progress_bar:
        train_loss = 0.0
        train_predictions = []
        train_targets = []
        val_predictions = []
        val_targets = []
        for batch in trainer.get_eval_dataloader():
            model_input = batch["input_ids"].to(trainer.model.device)
            targets = batch["labels"].to(trainer.model.device)

            outputs = trainer.model(model_input, labels=targets)
            loss = outputs.loss
            logits = outputs.logits

            train_loss += loss.item()
            train_predictions.append(logits.detach().cpu().numpy())
            train_targets.append(targets.detach().cpu().numpy())

        train_loss /= len(trainer.get_eval_dataloader())
        train_predictions = np.concatenate(train_predictions, axis=0)
        train_targets = np.concatenate(train_targets, axis=0)
        train_accuracy = (train_predictions.argmax(axis=1) == train_targets).mean()

        val_loss = trainer.evaluate().get("eval_loss", None)

        # Call trainer.predict() on the val dataset
        val_outputs = trainer.predict(dataset["val"])
        val_predictions = val_outputs.predictions
        val_targets = val_outputs.label_ids

        val_accuracy = (val_predictions.argmax(axis=1) == val_targets).mean()

        precision, recall, f1, _ = precision_recall_fscore_support(
            val_targets, val_predictions.argmax(axis=1), average="weighted"
        )

        print(f"Epoch: {epoch + 1}",f"Train Loss: {train_loss}",f"Val Loss: {val_loss}",f"Train Accuracy: {train_accuracy}",f"Val Accuracy: {val_accuracy}",f"Precision: {precision}",f"Recall: {recall}",f"F1: {f1}")

        wandb.log(
            {
                "Epoch": epoch + 1,
                "Train Loss": train_loss,
                "Val Loss": val_loss,
                "Train Accuracy": train_accuracy,
                "Val Accuracy": val_accuracy,
                "Precision": precision,
                "Recall": recall,
                "F1": f1,
            }
        )

    wandb.finish()

    return


In [ ]:
torch.cuda.empty_cache()

In [ ]:
from transformers import AdamW
from transformers import get_cosine_schedule_with_warmup
optimizer = AdamW(model.parameters(), lr=1e-5)
num_training_steps = 10
lr_scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.1*num_training_steps, num_training_steps=num_training_steps)
objective(model,0.0001,5,16,16, optimizer, lr_scheduler)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Epochs:  80%|████████  | 4/5 [11:34<02:53, 173.55s/it]

Epoch: 4 Train Loss: 0.19114227229807557 Val Loss: 0.0873590037226677 Train Accuracy: 0.9031155514657552 Val Accuracy: 0.9719994741685289 Precision: 0.9724378535888728 Recall: 0.9719994741685289 F1: 0.9720041771336818


Epochs: 100%|██████████| 5/5 [14:27<00:00, 173.60s/it]

Epoch: 5 Train Loss: 0.19114227229807557 Val Loss: 0.0873590037226677 Train Accuracy: 0.9031155514657552 Val Accuracy: 0.9719994741685289 Precision: 0.9724378535888728 Recall: 0.9719994741685289 F1: 0.9720041771336818


Epoch,▁▃▅▆█
F1,▁▁▁▁▁
Precision,▁▁▁▁▁
Recall,▁▁▁▁▁
Train Accuracy,▁████
Train Loss,█▁▁▁▁
Val Accuracy,▁▁▁▁▁
Val Loss,▁▁▁▁▁
eval/loss,▁▁▁▁▁
eval/runtime,▁██▇█
eval/samples_per_second,█▁▁▂▁


In [ ]:
model.save_pretrained('mBert_exp1/hf/')

In [ ]:
import transformers

In [ ]:
# Save the model weights
torch.save(model.state_dict(), "mBert_exp1/mBert-model.pt")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
